In [106]:
import pulp
import pandas as pd
from pulp import LpMaximize, LpProblem, LpVariable, LpBinary ,LpMinimize,lpSum
import random
from datetime import datetime

In [159]:
df = pd.read_csv("../data/shift_dataBase_testing.csv", converters={"staff_level": eval}) 

df.head()

staff = df["name"].unique()
 

staff_level_dict = {name: random.randint(1, 5) for name in staff}
staff_status_list = ["International Student", "High school student", "Fulltime", "Part Time"]

df["staff_level"] = df["name"].map(staff_level_dict)
df["staff_status"] = df["name"].map(staff_status_dict)


print(staff_level_dict)
display(df.head())

df = df.sort_values(by="date")
df["date"] = pd.to_datetime(df["date"])



FileNotFoundError: [Errno 2] No such file or directory: '../data/shift_dataBase_testing.csv'

In [108]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137 entries, 0 to 136
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   ID         137 non-null    int64 
 1   name       137 non-null    object
 2   date       137 non-null    object
 3   morning    137 non-null    bool  
 4   afternoon  137 non-null    bool  
 5   night      137 non-null    bool  
dtypes: bool(3), int64(1), object(2)
memory usage: 3.7+ KB


In [109]:

time_map = {
    "morning": list(range(9, 13)),     
    "afternoon": list(range(13, 18)), 
    "night": list(range(18, 25))       
}

expanded_rows = []


for _, row in df.iterrows():
    for shift in ["morning", "afternoon", "night"]:
        if row[shift] == True or row[shift] == 1:  
            for hour in time_map[shift]:
                expanded_rows.append({
                    "staff_id": row["staff_id"],
                    "name": row["name"],
                    "date": row["date"].strftime("%Y-%m-%d"),
                    "hour": hour,
                    "staff_level": row["staff_level"],
                    "staff_status" : row["staff_status"]
                    
                })

hourly_df = pd.DataFrame(expanded_rows)


KeyError: 'staff_id'

In [110]:
display(hourly_df)

NameError: name 'hourly_df' is not defined

In [111]:
model = LpProblem("ShiftOptimization", LpMinimize)
x = {}

# x[スタッフ, 日付, 時間] = 0 or 1（勤務かどうか）
for _, row in hourly_df.iterrows():
    key = (row["staff_id"], row["date"], row["hour"])
    x[key] = LpVariable(f"x_{row['staff_id']}_{row['date']}_{row['hour']}", cat="Binary")

NameError: name 'hourly_df' is not defined

In [112]:
hours = sorted(hourly_df["hour"].unique())[:len(pred)]


14,15,20,10,15,10,12]  # 各時間帯に必要なスタッフレベル合計

for i, hour in enumerate(hours):
    for date in hourly_df["date"].unique():
        terms = []
        for sid in hourly_df["staff_id"].unique():
            if (sid, date, hour) in x:
                staff_level_val = hourly_df.loc[
                    (hourly_df["staff_id"] == sid) &
                    (hourly_df["date"] == date) &
                    (hourly_df["hour"] == hour),
                    "staff_level"
                ]
                if not staff_level_val.empty:
                    level_value = staff_level_val.values[0]
                    if isinstance(level_value, list):
                        level_value = level_value[0]
                    level = float(level_value)
                    terms.append(x[sid, date, hour] * level)
        model += lpSum(terms) >= pred[i], f"NeedLevel_{date}_{hour}"


NameError: name 'hourly_df' is not defined

In [113]:
for _, row in hourly_df.iterrows():
    sid = row["staff_id"]
    date = row["date"]
    hour = row["hour"]
    status = row["staff_status"]

    if status == "High school student" and hour >= 22:
        if (sid, date, hour) in x:
            model += x[sid, date, hour] == 0, f"HighSchoolNoLate_{sid}_{date}_{hour}"


NameError: name 'hourly_df' is not defined

In [114]:
model += lpSum(x.values()), "MinimizeTotalShifts"


In [115]:
results = []
for (sid, date, hour), var in x.items():
    if var.varValue == 1:
        results.append({
            "staff_id": sid,
            "date": str(date),  # ←ここを追加！
            "hour": hour
        })

if len(results) == 0:
    print("⚠️ 結果が空です。モデルに解があるか、制約が厳しすぎないか確認してください。")
else:
    result_df = pd.DataFrame(results).sort_values(by=["date", "hour", "staff_id"])
    print(result_df.head())


⚠️ 結果が空です。モデルに解があるか、制約が厳しすぎないか確認してください。


In [116]:
from pulp import LpStatus
print("モデルの状態:", LpStatus[model.status])


モデルの状態: Not Solved


In [117]:
from pulp import PULP_CBC_CMD, LpStatus, value

# 最後に solve() を忘れずに！
model.solve(PULP_CBC_CMD(msg=True))  # msg=True でログ出力も見れる

# ステータス確認
print("モデルの状態:", LpStatus[model.status])

# 結果が得られている場合のみ表示
if model.status == 1:
    results = []
    for (sid, date, hour), var in x.items():
        if var.varValue == 1:
            results.append({
                "staff_id": sid,
                "date": str(date),
                "hour": hour
            })
    result_df = pd.DataFrame(results).sort_values(by=["date", "hour", "staff_id"])
    display(result_df.head())
else:
    print("❌ モデルは解かれていません。制約条件やデータを見直してください。")


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pulp/apis/../solverdir/cbc/osx/i64/cbc /var/folders/23/dxqwsqmd1w3g02ljvb96m6fm0000gn/T/dd2ef3f2b4a1492aa089ddd7b62dd7cf-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/23/dxqwsqmd1w3g02ljvb96m6fm0000gn/T/dd2ef3f2b4a1492aa089ddd7b62dd7cf-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 5 COLUMNS
At line 7 RHS
At line 8 BOUNDS
At line 10 ENDATA
Problem MODEL has 0 rows, 1 columns and 0 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Empty problem - 0 rows, 1 columns and 0 elements
Optimal - objective value 0
Optimal objective 0 - 0 iterations time 0.002
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.01

モデルの状態: Optimal


KeyError: 'date'

In [118]:
x = {}
for _, row in hourly_df.iterrows():
    key = (row["staff_id"], row["date"], row["hour"])
    x[key] = LpVariable(f"x_{row['staff_id']}_{row['date']}_{row['hour']}", cat="Binary")


NameError: name 'hourly_df' is not defined

In [119]:
Help = {}
for date in hourly_df["date"].unique():
    for hour in hourly_df["hour"].unique():
        Help[date, hour] = LpVariable(f"Help_{date}_{hour}", cat="Binary")


NameError: name 'hourly_df' is not defined

In [120]:
model += lpSum(Help[date, hour] for date in hourly_df["date"].unique()
                                 for hour in hourly_df["hour"].unique())


NameError: name 'hourly_df' is not defined

In [121]:
for i, hour in enumerate(hours):  # pred[i]に対応
    for date in hourly_df["date"].unique():
        terms = []
        for sid in hourly_df["staff_id"].unique():
            key = (sid, date, hour)
            if key in x:
                staff_level_val = hourly_df[
                    (hourly_df["staff_id"] == sid) &
                    (hourly_df["date"] == date) &
                    (hourly_df["hour"] == hour)
                ]["staff_level"].values
                if len(staff_level_val) > 0:
                    level = float(staff_level_val[0])
                    terms.append(x[key] * level)

      
        model += lpSum(terms) + 100 * Help[date, hour] >= pred[i], f"LevelOrHelp_{date}_{hour}"


NameError: name 'hours' is not defined

In [122]:
for key, var in Help.items():
    if var.varValue == 1:
        print(f"⚠️ Help needed on {key}")


In [123]:
results = []

for (sid, date, hour), var in x.items():
    if var.varValue == 1:
        results.append({
            "staff_id": sid,
            "date": date,
            "hour": hour,
            "assigned": 1
        })


In [124]:
help_needed = []

for (date, hour), var in Help.items():
    if var.varValue == 1:
        help_needed.append({
            "date": date,
            "hour": hour,
            "HELP": 1
        })




In [125]:
df_temp = pd.DataFrame(results)
print(df_temp.columns)  # ['staff_id', 'date', 'hour', 'assigned'] があるか確認

# そのあとに並び替え
if all(col in df_temp.columns for col in ["date", "hour", "staff_id"]):
    result_df = df_temp.sort_values(by=["date", "hour", "staff_id"])
else:
    print("❌ 必要なカラムが足りません。")


RangeIndex(start=0, stop=0, step=1)
❌ 必要なカラムが足りません。


In [160]:
staff_pro_df = pd.read_csv("../data/__________20_____.csv")
staff_pro_df.head()
staff_status = staff_pro_df["status"].unique()
staff_pro_df["status"] = staff_pro_df["status"].apply(lambda x: "International Student" if x == "留学生" else x)
pred = [10,15,25,10,12,14,10]

In [161]:
display(staff_status)

array(['留学生', 'International Student', 'Regular', 'Part-time'],
      dtype=object)

In [162]:
staff_pro_df.head()


,ID,Name,Level,Gender,Age,Email,status
0,1001,khh,3,Male,24,khh@gmail.com,International Student
1,1002,kph,3,Female,24,kph,International Student
2,1003,yss,3,Male,24,yss@gmail.com,International Student
3,1004,user1004,1,Female,28,user1004@example.com,Regular
4,1005,user1005,5,Female,28,user1005@example.com,International Student


In [130]:
df = pd.read_csv("../data/_________.csv")

In [131]:
df.head()
df = df.rename(columns={"staff_id":"ID"})

In [132]:
final_df = pd.merge(staff_pro_df,df,on="ID")

In [164]:
final_df.head()
final_df.to_csv("../data/testing.csv")

In [134]:
final_df = final_df.drop('name', axis=1)

In [135]:
final_df.head()

,ID,Name,Level,Gender,Age,Email,status,date,morning,afternoon,night
0,1001,khh,3,Male,24,khh@gmail.com,International Student,2025-07-01,True,True,False
1,1001,khh,3,Male,24,khh@gmail.com,International Student,2025-07-02,False,True,True
2,1001,khh,3,Male,24,khh@gmail.com,International Student,2025-07-03,True,True,True
3,1001,khh,3,Male,24,khh@gmail.com,International Student,2025-07-04,False,True,False
4,1001,khh,3,Male,24,khh@gmail.com,International Student,2025-07-05,False,True,True


In [136]:
time_map = {
    "morning": list(range(9, 13)),     
    "afternoon": list(range(13, 18)),   
    "night": list(range(18, 25))       
}

In [137]:
prob = LpProblem("Shift_Optimization", LpMinimize)

In [151]:
shift_var = {}
for index, row in df.iterrows():
    for time in ["morning", "afternoon", "night"]:
        if row[time] == 1:  
            var_name = f"x_{row["ID"]}_{row['date']}_{time}"
            shift_var[var_name] = LpVariable(var_name, 0, 1, cat="Binary")

In [152]:
for staff_id in final_df["ID"].unique():
    for date in final_df["date"].unique():
        relevant = [var for var in shift_var if f"{"ID"}_{date}" in var]
        prob += lpSum([shift_var[v] for v in relevant]) <= 1


In [153]:
prob += lpSum(shift_var.values())


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pulp/pulp.py:1831: UserWarning: Overwriting previously set objective.
  warnings.warn("Overwriting previously set objective.")


In [154]:
prob.solve()


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pulp/apis/../solverdir/cbc/osx/i64/cbc /var/folders/23/dxqwsqmd1w3g02ljvb96m6fm0000gn/T/fb48f807a6934eabbce72bc2e42dd7e4-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/23/dxqwsqmd1w3g02ljvb96m6fm0000gn/T/fb48f807a6934eabbce72bc2e42dd7e4-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 327 COLUMNS
At line 1870 RHS
At line 2193 BOUNDS
Bad image at line 2195 <  BV BND       X0000001 >
Bad image at line 2197 <  BV BND       X0000003 >
No match for column X0000004 at line 2198 <  BV BND       X0000004 >
Bad image at line 2200 <  BV BND       X0000006 >
No match for column X0000007 at line 2201 <  BV BND       X0000007 >
Bad image at line 2203 <  BV BND       X0000009 >
Bad image at line 2205 <  BV BND       X0000011 >
Bad image at line 2207 <  BV BND       X0

PulpSolverError: Pulp: Error while executing /Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pulp/apis/../solverdir/cbc/osx/i64/cbc

In [156]:
results = []
for name, var in shift_var.items():
    if var.value() == 1:
        parts = name.split("_")
        results.append({"ID": parts[1], "date": parts[2], "shift": parts[3]})

result_df = pd.DataFrame(results)


In [157]:
print(result_df)

Empty DataFrame
Columns: []
Index: []
